# ECT of leaf embedded graphs

#### Sarah McGuire October, 2023

Import necessary packages

In [1]:
import importlib
import time

import numpy as np
import pandas as pd
import math
import seaborn as sns

import scipy.ndimage as ndimage
import scipy.spatial.distance as distance

%matplotlib inline
from matplotlib import pyplot as plt
import matplotlib.gridspec as gridspec

import IPython.display as display

import networkx as nx

# To import/export
from PIL import Image


Read in leaf landmark coordinates file

In [2]:
filename = "../data/0.procrustes_landmarks.txt"

In [3]:
df = pd.read_csv(filename, sep='	').transpose()

In [4]:
df.head()

,0,1,2,3,4,5,6,7,8,9,...,3309,3310,3311,3312,3313,3314,3315,3316,3317,3318
plant,Pact1,Pact1,Pact1,Pact1,Pact1,Pact1,Pact1,Pact1,Pact1,Pact1,...,Pvil4,Pvil4,Pvil4,Pvil4,Pvil4,Pvil4,Pvil4,Pvil4,Pvil4,Pvil4
species,actinia,actinia,actinia,actinia,actinia,actinia,actinia,actinia,actinia,actinia,...,villosa,villosa,villosa,villosa,villosa,villosa,villosa,villosa,villosa,villosa
ontogeny,1,2,3,4,5,6,7,8,9,10,...,6,7,8,9,10,11,12,13,14,15
total,17,17,17,17,17,17,17,17,17,17,...,15,15,15,15,15,15,15,15,15,15
heteroblasty,17,16,15,14,13,12,11,10,9,8,...,10,9,8,7,6,5,4,3,2,1


In [34]:
df[17]

plant                 Pact2
species             actinia
ontogeny                  1
total                    14
heteroblasty             14
x1               -44.918266
y1               393.241955
x2               -56.996588
y2               374.975493
x3               -26.231109
y3               360.611771
x4                28.656493
y4               325.944374
x5                52.375182
y5                354.61086
x6                37.966676
y6               379.104438
x7               -448.03707
y7               243.830209
x8              -480.335041
y8               203.018901
x9              -539.171966
y9              -393.629227
x10             -533.094556
y10             -453.969057
x11              -67.122271
y11            -1544.337476
x12              581.284358
y12             -403.004684
x13              599.540556
y13             -302.987351
x14              469.789777
y14              196.915193
x15              426.293825
y15              265.674601
Name: 17, dtype: obj

In [6]:
df.iloc[0]

0       Pact1
1       Pact1
2       Pact1
3       Pact1
4       Pact1
        ...  
3314    Pvil4
3315    Pvil4
3316    Pvil4
3317    Pvil4
3318    Pvil4
Name: plant, Length: 3319, dtype: object

Save the labels as a separate csv for use in CNN

In [7]:
labels = df[:][:5].transpose()
labels.to_csv('../data/labels.csv')

Build a (not necessarily embedded) graph of all $15$ vertices and $24$ edges.

In [8]:
G = nx.Graph()
G.add_edge(1, 6)
G.add_edge(1, 7)
G.add_edge(1, 2)
G.add_edge(2, 7)
G.add_edge(2, 9)
G.add_edge(2, 3)
G.add_edge(3, 9)
G.add_edge(3, 11)
G.add_edge(3, 4)
G.add_edge(4, 11)
G.add_edge(4, 13)
G.add_edge(4, 5)
G.add_edge(5, 13)
G.add_edge(5, 15)
G.add_edge(5, 6)
G.add_edge(6, 15)
G.add_edge(7, 8)
G.add_edge(8, 9)
G.add_edge(9, 10)
G.add_edge(10, 11)
G.add_edge(11, 12)
G.add_edge(12, 13)
G.add_edge(13, 14)
G.add_edge(14, 15)

### Function to compute ECC for a given direction

In [6]:
def ECC(G, pos, theta, numThresh):
    omega = (np.cos(theta), np.sin(theta))
    
    # list of vertices and vertex positions
    v_list = list(pos.keys())
    pos_list = list(pos.values())
    
    # function g 
    def g(v): 
        return np.dot(pos_list[v-1],omega)
    # sort the v_list using g(v)
    v_list.sort(key=g, reverse = True) 
    
        
    # function to compute the number of lower edges of a vertex $v$
    # for a specific direction (included by the use of sorted v_list)
    def lower_edges(v):
        L = [n for n in G.neighbors(v)]
        Lg = [g(v) for v in L]
        return sum(n > g(v) for n in Lg)
    
    # find a bounding box of points
    def bounding_box(points):
        x_coord, y_coord = zip(*points)
        return [(min(x_coord), min(y_coord)), (max(x_coord), max(y_coord))]

    x, y = zip(*pos_list)
    x_box,y_box = zip(*bounding_box(pos_list))
    # bounding box size (use to get a radius for the bounding circle)
    dist = (math.dist(bounding_box(pos_list)[0],bounding_box(pos_list)[1]))
    r = dist/2
    # bounding circle midpoint
    midpt = ((x_box[0]+x_box[1])/2, (y_box[0]+y_box[1])/2)
    
    r_threshes = np.linspace(r, -r, numThresh)
    g_thresh=[]
    for r_thresh in r_threshes:
        pt = (midpt[0]+ r_thresh*omega[0], midpt[1]+ r_thresh*omega[1])
        g_thresh.append(np.dot(pt,omega))

   

    # Full ECC vector
    ecc = []
    ecc.append(0)
    v = 0 # 
    for i in range(1,numThresh+1):
        if v==len(v_list):
            ecc.append(ecc[i-1])
        elif (g_thresh[i-1] < g(v_list[v])) & (v<len(v_list)):
            x = ecc[i-1] #previous value of ECC
            k = lower_edges(v_list[v])
            x+=1 #add 1 to vertex count
            x-=k #subtract the the number of lower edges
            ecc.append(x)
            v+=1 # move to the next vertex in v_list
        else:
            ecc.append(ecc[i-1])
    ecc = ecc[1:] #Drop the initial 0 value   
    
    
    return ecc


FOR each leaf...
- Get the vertex positions from df (pos)
- Compute ECT
- Save ECT matrix
    

In [27]:
dataset = []
for p in range(len(df.columns)):
    
    # Get the vertex positions
    pos = {}
    valuesX = df[p].iloc[5::2]
    valuesY = df[p].iloc[6::2]
    for i in range(15):
        pos[i+1] = (valuesX[i],valuesY[i])
        
    # Select directions around the circle
    numCircleDirs = 32
    circledirs =  np.linspace(0, 2*np.pi, num=numCircleDirs, endpoint=False)
    
    # Choose number of thresholds for the ECC
    numThresh = 48
    
    # Compute the ECT of leaf p for numCircleDirs, numThresh
    ECT_preprocess = {}
    for i, angle in enumerate(circledirs):

        outECC = ECC(G, pos, theta=angle, numThresh = numThresh)

        ECT_preprocess[i] = (angle,outECC)

    # Making a matrix M[i,j]: (numThresh x numCircleDirs)
    M = np.empty([numThresh,numCircleDirs])
    for j in range(M.shape[1]):
        M[:,j] = ECT_preprocess[j][1]
        
        
    # Rescale the array pixel values to [0, 255]-- > wait until data loader transform for Normalization
    # This is necessary to save as grayscale image (because the ECT has negative integer values)
    M_scale = (255*(M - np.min(M))/np.ptp(M)).astype(np.uint8)
    

    # Convert to greyscale image using PIL
    M_img = Image.fromarray(M_scale)


    # PNG file to save
    #filename = 'data_ECT/'+str(numCircleDirs)+'dirs_'+str(numThresh)+'thresh/leaf_'+str(p)+'.png'
    #M_img.save(filename, "PNG")
    
    # NPY file to save
    filename = 'data_ECT/'+str(numCircleDirs)+'dirs_'+str(numThresh)+'thresh_NPY/leaf_'+str(p)
    np.save(filename, M)
    
    # flatten array and append to dataset list for statistics computation of entire dataset
    dataset.append(M.flatten())


In [36]:
# calculate global min, max, average, and standard deviation of dataset for normalization later
global_mean = np.mean(dataset)
global_std = np.std(dataset)
global_min = np.min(dataset)
global_max = np.max(dataset)

In [38]:
print('----------------')
print('Dataset statistics')
print('----------------')
print('min:', global_min)
print('max:', global_max)
print('mean:', global_mean)
print('std:', global_std)

----------------
Dataset statistics
----------------
min: -9.0
max: 5.0
mean: -2.377006087112082
std: 3.7091372538232994


In [12]:
for p in range(len(df.columns)):
    valuesX = df[p].iloc[5::2]
    valuesY = df[p].iloc[6::2]
    print(valuesX)

x1       1.939415
x2      -7.047279
x3      28.429685
x4      65.726986
x5      99.227324
x6      88.467463
x7    -497.008526
x8    -517.057806
x9    -543.085079
x10   -531.777733
x11   -108.831734
x12    563.252807
x13    570.669793
x14    405.345149
x15    381.749535
Name: 0, dtype: object
x1     -27.972095
x2     -27.127051
x3     -17.138482
x4      21.438983
x5      42.800256
x6       33.87919
x7    -461.470174
x8    -506.624409
x9    -563.794896
x10    -543.71884
x11     28.713845
x12    525.022693
x13    538.426886
x14    489.936964
x15    467.627131
Name: 1, dtype: object
x1     -43.242816
x2     -39.277936
x3     -34.239161
x4       4.770596
x5      19.758407
x6      14.307904
x7    -472.831464
x8    -500.030931
x9     -568.30201
x10   -555.293344
x11    120.698695
x12    546.694706
x13    557.727421
x14    488.911001
x15    460.348932
Name: 2, dtype: object
x1     -36.082509
x2     -30.527203
x3     -19.829324
x4      13.246641
x5      25.471213
x6       23.49581
x7    -470.33

x1      -17.230133
x2      -28.695903
x3      -19.962141
x4        0.345472
x5        8.410276
x6        2.132846
x7     -456.203535
x8     -520.313654
x9    -1062.394414
x10    -135.051108
x11      41.773999
x12     135.882244
x13    1062.944428
x14     526.119316
x15     462.242309
Name: 1241, dtype: object
x1        -9.73869
x2      -14.254844
x3       -10.09468
x4       13.324924
x5       19.356999
x6       17.799113
x7     -568.255429
x8     -634.339722
x9    -1039.593192
x10    -194.648785
x11      36.344321
x12      178.67326
x13     1032.00781
x14     643.032923
x15     530.385992
Name: 1242, dtype: object
x1      -37.306134
x2      -42.758574
x3      -34.297605
x4       -1.134771
x5        6.201179
x6       -2.243275
x7     -523.350492
x8     -575.299755
x9    -1031.535375
x10    -196.764848
x11      11.366143
x12     214.478012
x13    1064.234302
x14     608.955663
x15      539.45553
Name: 1243, dtype: object
x1      -14.744894
x2      -24.345413
x3      -19.505967
x4       1

x1      -2.860053
x2      13.570155
x3      38.888307
x4      95.251214
x5      89.420508
x6      83.232855
x7    -549.266137
x8      -626.4711
x9    -711.936201
x10   -205.316359
x11   -110.336168
x12    275.375433
x13    676.839328
x14    498.275264
x15    435.332955
Name: 2161, dtype: object
x1     -43.504441
x2     -29.594238
x3      -29.65096
x4      41.781779
x5      64.149987
x6      65.808725
x7    -483.469992
x8    -576.230156
x9    -642.862538
x10   -453.605797
x11    -53.721879
x12    356.980177
x13    717.757863
x14    578.920375
x15    487.241094
Name: 2162, dtype: object
x1     -51.747613
x2     -72.675565
x3     -35.914166
x4      14.873199
x5      61.540035
x6      45.464182
x7     -274.34113
x8     -332.99322
x9    -649.423708
x10   -128.913363
x11   -171.817245
x12     44.815506
x13    956.596214
x14    316.558495
x15    277.978378
Name: 2163, dtype: object
x1      -56.330009
x2      -68.554777
x3      -30.279006
x4        0.607454
x5       60.437406
x6        47.3510

x1     -25.909303
x2      -35.78841
x3     -15.118731
x4      13.971482
x5       39.86879
x6      33.124499
x7    -697.606494
x8    -763.159577
x9     -876.24007
x10   -312.002663
x11     10.051151
x12    317.796636
x13    887.659478
x14    753.374448
x15    669.978764
Name: 3002, dtype: object
x1     -18.820011
x2     -28.300986
x3     -14.032031
x4      17.512047
x5      36.655013
x6      28.035068
x7    -728.823843
x8    -764.241126
x9    -876.168392
x10   -326.534373
x11     21.888119
x12    348.525407
x13    866.426216
x14    748.331697
x15    689.547195
Name: 3003, dtype: object
x1     -42.176755
x2     -34.791622
x3       -33.9732
x4      -1.513253
x5      15.022434
x6        5.42176
x7    -683.825448
x8    -738.217437
x9     -797.82344
x10    -363.66683
x11     39.876429
x12    357.779347
x13    801.320921
x14     759.73125
x15    716.835844
Name: 3004, dtype: object
x1     -16.711145
x2     -11.847228
x3      -4.251305
x4      22.656372
x5      42.123198
x6      33.657845
x7  

In [35]:
classes = df.T['species'].unique().tolist()
print(classes)
df[p].species

['actinia', 'alata', 'amethystina', 'biflora', 'caerulea', 'capsularis', 'cincinnata', 'coccinea', 'coriacea', 'cristalina', 'edmundoi', 'edulis', 'foetida', 'galbana', 'gibertii', 'gracilis', 'hatschbachii', 'kermesina', 'ligularis', 'maliformis', 'malacophylla', 'micropetala', 'miersii', 'miniata', 'misera', 'mollissima', 'morifolia', 'mucronata', 'nitida', 'organensis', 'pohlii', 'racemosa', 'rubra', 'setacea', 'sidifolia', 'suberosa', 'tenuifila', 'triloba', 'tricuspis', 'villosa']


'villosa'